In [1]:
import pandas as pd
import numpy as np
import pickle
#import shap
import os
import time
import builtins
import sys

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge as Model
from sklearn.model_selection import GridSearchCV

/opt/modules/i12g/anaconda/envs/jupyterhub/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
project_dir = '/s/project/mll/sergey/pausing/'

In [3]:
output_dir = project_dir + 'linear_regression'

os.makedirs(output_dir, exist_ok=True)

In [4]:
hepg2_df = pd.read_csv(project_dir + 'data/individual.HepG2.all.features.csv', index_col=0)

In [5]:
k562_df = pd.read_csv(project_dir + 'data/individual.K562.all.features.csv', index_col=0)

In [6]:
def print(*args, **kwargs):
    '''
    Redefine print function for logging
    '''
    now = time.strftime("[%Y/%m/%d-%H:%M:%S]-", time.localtime()) #place current time at the beggining of each printed line
    builtins.print(now, *args, **kwargs)
    sys.stdout.flush()

In [7]:
def clean_data(df):

    df = df.fillna(df.median())

    df.loc[np.isfinite(df['tx.ex.ratio'])==False,'tx.ex.ratio'] = 1
        
    return df

In [8]:
def normalize_features(X_train, X_test):

    zero_variance_columns = X_train.columns[(X_train.std()==0).values]
    
    X_train = X_train.drop(columns=zero_variance_columns)
    X_test = X_test.drop(columns=zero_variance_columns)

    mean = X_train.mean()
    std = X_train.std()
    
    X_train_norm = (X_train-mean)/std
    
    X_test_norm = (X_test-mean)/std
    
    return X_train_norm, X_test_norm

In [9]:
def hpp_search(X_train, y_train):
    
    model = Model(random_state=1)

    parameters = {'alpha':np.arange(1,10000,100)}

    clf = GridSearchCV(model, parameters, verbose=0)

    clf.fit(X_train, y_train)
    
    return clf

In [10]:
def save_model(model, output_name):
    
    output_path = output_dir + '/' + output_name + '.pickle'
    
    with open(output_path, 'wb') as f:
        pickle.dump(model,f)
        
    print('Model saved to: ', output_path)

In [58]:
def compute_shap_values(model, X_test, output_name):
    
    print('Computing Shapley values...')
    
    #X100 = shap.utils.sample(X_test, 100)

    #explainer = shap.Explainer(model.predict, X100, max_evals = 2 * X100.shape[1] + 1)
    
    #shap_values = explainer(X_test, silent=True)
    
    shap_values = (model.coef_ * X_test) - (model.coef_ * X_test).mean()
    
    shap_values = pd.DataFrame(shap_values.values, index=X_test.index, columns=X_test.columns)

    ##proteins = [x.split('.')[1] if 'chip' in x or 'clip' in x else 'NA' for x in X_test.columns]

    ##shap_df = pd.DataFrame({'protein':proteins, 'shap_values_sum':np.sum(shap_values.values,axis=0)})

    ##shap_df.groupby('protein').sum().to_csv(output_dir + '/' + output_name + '.csv')
    
    output_path = output_dir + '/' + output_name + '.csv.gz'
    
    shap_values.to_csv(output_path)
    
    print('Shapley values saved to: ', output_path)

In [11]:
data = {'HepG2':clean_data(hepg2_df), 'K562':clean_data(k562_df)}

target_column = 'target_traveling.ratio'

In [12]:
best_params = {'HepG2':{'alpha':2800},'K562':{'alpha':2800}}

In [14]:
for cell_line in ('K562',):

    print('First evaluation, 50/50 train/test split:', cell_line)
    
    df = data[cell_line]
    
    y = df[target_column]
    
    X = df.drop(columns=target_column)
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.5, random_state=1)
    
    X_train, X_test = normalize_features(X_train, X_test)
    
    if len(best_params[cell_line])==0:
    
        print('Looking for optimal hyperparameters...')

        model = hpp_search(X_train, y_train)
        
        best_params[cell_line] = model.best_params_
        
        save_model(model,'train_on_half_' + cell_line + '_hpp_search' )
        
    else:
        
        model = Model(**best_params[cell_line],random_state=1)
        
        model.fit(X_train, y_train)
        
    print('Using hyperparameters: ', best_params[cell_line])
    
    train_score = model.score(X_train, y_train)
    
    test_score = model.score(X_test, y_test)

    y_pred = model.predict(X_test)

    print('Train score: ', round(train_score,3))

    print('Test score: ', round(test_score,3))
    
    #compute_shap_values(model, X_test, 'train_on_' + cell_line + '_test_on_' + cell_line + '_shapley_values')
    
    print()

[2023/06/06-20:55:09]- First evaluation, 50/50 train/test split: K562
[2023/06/06-20:55:10]- Using hyperparameters:  {'alpha': 2800}
[2023/06/06-20:55:10]- Train score:  0.739
[2023/06/06-20:55:10]- Test score:  0.601
[2023/06/06-20:55:10]-


In [15]:
#with open(project_dir + '/K562_50_50_preds/' + 'linear_regression.pickle', 'wb') as f:
#     pickle.dump({'y_true':y_test, 'y_pred':y_pred},f)

In [48]:
common_features = set(data['HepG2'].columns).intersection(set(data['K562'].columns))
common_features.remove(target_column)

In [49]:
common_genes = set(data['HepG2'].index).intersection(set(data['K562'].index))

unique_genes = {'HepG2':set(data['HepG2'].index).difference(set(data['K562'].index)),
               'K562':set(data['K562'].index).difference(set(data['HepG2'].index))}

In [50]:
best_params_common = {'HepG2':{'alpha':2800},'K562':{'alpha':2800}}

In [13]:
for train_cell_line, test_cell_line in (('HepG2','K562'), ('K562','HepG2')):

    print('Second evaluation, train on ', train_cell_line,' evaluate on ', test_cell_line )
    
    train_df = data[train_cell_line]
    
    y_train = train_df[target_column]
    X_train = train_df[common_features]
    
    test_df = data[test_cell_line]
    
    y_test = test_df[target_column]
    X_test = test_df[common_features]
    
    X_train, X_test = normalize_features(X_train, X_test)
    
    if len(best_params_common[train_cell_line])==0:
    
        print('Looking for optimal hyperparameters...')
    
        model = hpp_search(X_train, y_train)
        
        best_params_common[train_cell_line] = model.best_params_
        
        save_model(model,'train_on_full_' + train_cell_line + '_hpp_search' )

    else:
        
        model = Model(**best_params_common[train_cell_line],random_state=1)

        model.fit(X_train, y_train)
        
    print('Using hyperparameters: ', best_params_common[train_cell_line])
    
    train_score = model.score(X_train, y_train)
    
    test_score = model.score(X_test, y_test)
    
    print('Train score: ', round(train_score,3))

    print('Test score: ', round(test_score,3))
    
    test_score_common = model.score(X_test.loc[X_test.index.isin(common_genes)], y_test.loc[y_test.index.isin(common_genes)])

    print('Test score on common genes: ', round(test_score_common,3))
    
    test_score_unique = model.score(X_test.loc[X_test.index.isin(unique_genes[test_cell_line])], y_test.loc[y_test.index.isin(unique_genes[test_cell_line])])

    print('Test score on genes unique for the test cell line', '('+test_cell_line+'):', round(test_score_unique,3))
    
    print()

Second evaluation, train on  HepG2  evaluate on  K562
Using best hyperparameters:  {'alpha': 2800}
Train score:  0.603
Test score:  0.422
Test score on common genes:  0.241
Test score on genes unique for the test cell line (K562): 0.477

Second evaluation, train on  K562  evaluate on  HepG2
Using best hyperparameters:  {'alpha': 2800}
Train score:  0.664
Test score:  0.445
Test score on common genes:  0.377
Test score on genes unique for the test cell line (HepG2): 0.451

